<a href="https://colab.research.google.com/github/GuoZhuyin/zhuyinasia/blob/main/DeepLearning/trainmodel_TEAM_855_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 訓練模型

In [ ]:
import PIL
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torchvision
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

In [ ]:
num_classes = 17

In [ ]:
from torchvision import models
model = models.inception_v3(pretrained=False)
model.aux_logits = False
model.fc =  nn.Linear(model.fc.in_features, num_classes) #調整輸出類別
# model = nn.DataParallel(model) #cuda使用

model=models.efficientnet_b0(pretrained=False, num_classes=num_classes)

model=models.efficientnet_b2(pretrained=False, num_classes=num_classes)

In [ ]:
model.load_state_dict(torch.load('../model/fbl_pt1-E60.pth')) #transfer learning

輸入的模型參數

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(299),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                   std=[0.229, 0.224, 0.225])
])

In [ ]:
# import torchvision.models as models
# num_classes = 17
# model=models.efficientnet_b0(pretrained=False, num_classes=num_classes)
# model.cuda()

In [ ]:
# train_transform = transforms.Compose([
#     #transforms.RandomResizedCrop(224),
#     #transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ])

In [ ]:
train_data = datasets.ImageFolder('farmlandtrain_imageV2', train_transform)

train_data = datasets.ImageFolder('farmlandtrain_imageV3', train_transform)

train_data = datasets.ImageFolder('farmlandtrain_imageV4', train_transform)

In [ ]:
num_classes=len(train_data.classes)
print(num_classes)

In [ ]:
train_data.class_to_idx

In [ ]:
batch_size = 64 #GPU batch
train_loader = DataLoader(train_data,batch_size=batch_size, shuffle=True)

In [ ]:
def imshow(img, title):
    img = torchvision.utils.make_grid(img, normalize=True)
    npimg = img.numpy()
    fig = plt.figure(figsize = (5, 15))
    plt.imshow(np.transpose(npimg,(1,2,0)))
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
imshow(images, [train_data.classes[i] for i in labels])

In [ ]:
model.training

In [ ]:
model = model.cuda()

In [ ]:
# loss = nn.CrossEntropyLoss()
# optimizer = torch.optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_epochs = 60 #迴圈

In [ ]:
for epoch in range(num_epochs):
    total_batch = len(train_data)//batch_size
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        # Zero your gradients for every batch!
        optimizer.zero_grad() #每次進行優化器前梯度歸零
        inputs = batch_images.cuda()
        labels = batch_labels.cuda()
        # Make predictions for this batch
        outputs  = model(inputs)
        
        # Compute the loss and its gradients
        cost = loss(outputs , labels)
        cost.backward()
        # Adjust learning weights
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], lter [{i+1}/{total_batch}] Loss: {cost.item():.4f}')

In [ ]:
torch.save(model.state_dict(), 'InV3_TEAM_855_5.0.pth') #model name

模型儲存名稱為InV3_TEAM_855_5.0.pth